In [1]:
import tensorflow as tf

In [2]:
import pandas as pd
import numpy as np

In [3]:
missings = pd.read_csv("../../dmbi-18-ddi/missings.csv", index_col=0)

In [4]:
tf.reset_default_graph()

In [ ]:
import time

In [9]:
dirs = ["1525391360"]
batch_size = 10000
final_preds = np.zeros(len(missings))
for d in dirs:
    #exec(open("{0}/tf_model.py".format(d)).read())
    tf.reset_default_graph()
    with tf.Session() as sess:
        saver = tf.train.import_meta_graph("./{0}/meta.model".format(d))
        saver.restore(sess, "./{0}/model.model".format(d))
        preds = np.array([])
        for i in range(0, len(missings), batch_size):
            t_preds =  sess.run("output:0", feed_dict={
                "left_index:0" : missings.node1.values[i:i+batch_size],
                "right_index:0" : missings.node2.values[i:i+batch_size]
            })
            preds = np.hstack([preds, t_preds])
    final_preds = final_preds + preds
final_preds = final_preds / len(dirs)

In [12]:
def write_solution_to_file(preds,file_path):
    #preds is assumed to be ordered by confidence level
    #adds the header to the soultion, combines the node IDs and writes the solution to file
    #asserts are important. Note them.

    print('writing predictions to file: {file_path}')
    for u, v in preds:
       assert u < v, 'graph is undirected, predict edges where the first node id is smaller than the second only'
    assert len(preds) == (1434 * 1434 - 1434 - 93200) / 2, "number of predictions is equal to number of non existing edges"
    #add title & predictions
    output = [','.join(["node1" + '_' + "node2"])]+[','.join([str(p[0]) +'_' + str(p[1])]) for p in preds]
    print(len(output))
    write_list_to_file(output,file_path)
    
def write_list_to_file(list, path):
    thefile = open(path, 'w')
    for item in list:
        thefile.write("%s\n" % item)
    thefile.close()


In [13]:
ordered_preds = missings.iloc[np.argsort(final_preds)[::-1]].T.apply(lambda x: (x.node1, x.node2))
write_solution_to_file(ordered_preds,'predictions_ensemble_{0}.csv'.format(int(time.time())))

NameError: name 'time' is not defined